#06_SparkDataAnal.ipynb<br>
TLC Trip Record Data<br>
출처: https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("spark-sql").getOrCreate()

In [3]:
df = spark.read.format('json')\
        .load("learning_spark_data/2015-summary.json")

In [18]:
df.count()

256

In [5]:
df.dtypes

[('DEST_COUNTRY_NAME', 'string'),
 ('ORIGIN_COUNTRY_NAME', 'string'),
 ('count', 'bigint')]

In [6]:
df.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)



In [7]:
df.collect()

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344),
 Row(DEST_COUNTRY_NAME='Egypt', ORIGIN_COUNTRY_NAME='United States', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='India', count=62),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Singapore', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Grenada', count=62),
 Row(DEST_COUNTRY_NAME='Costa Rica', ORIGIN_COUNTRY_NAME='United States', count=588),
 Row(DEST_COUNTRY_NAME='Senegal', ORIGIN_COUNTRY_NAME='United States', count=40),
 Row(DEST_COUNTRY_NAME='Moldova', ORIGIN_COUNTRY_NAME='United States', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Sint Maarten', count=325),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Marshall Islands', count=39),
 

In [17]:
# 도착 국가명 중복 제거 
df.select("DEST_COUNTRY_NAME").distinct().show(5)

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|         Anguilla|
|           Russia|
|         Paraguay|
|          Senegal|
|           Sweden|
+-----------------+
only showing top 5 rows



In [20]:
df1 = df.select('DEST_COUNTRY_NAME').distinct().cache()
df1.count()

132

In [22]:
# ROW class를 이용한 단일 레코드 생성
from pyspark.sql import Row
myRow = Row('hello', None, 1, False)
myRow

<Row('hello', None, 1, False)>

In [23]:
# 새로운 컬럼 추가하기
from pyspark.sql.functions import expr

df3 =df.withColumn('withincountry', expr('ORIGIN_COUNTRY_NAME==DEST_COUNTRY_NAME')) #expr sql표현식을 받아 생성 
df3 

DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: bigint, withincountry: boolean]

In [31]:
df3.show(3)

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withincountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
|    United States|            Ireland|  344|        false|
+-----------------+-------------------+-----+-------------+
only showing top 3 rows



In [35]:
df3.filter(df3.withincountry == True).show()

+-----------------+-------------------+------+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME| count|withincountry|
+-----------------+-------------------+------+-------------+
|    United States|      United States|370002|         true|
+-----------------+-------------------+------+-------------+



In [36]:
df3.filter(expr('withincountry == True')).show()

+-----------------+-------------------+------+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME| count|withincountry|
+-----------------+-------------------+------+-------------+
|    United States|      United States|370002|         true|
+-----------------+-------------------+------+-------------+



In [44]:
# 강사님 버전
df4 = df.withColumn('Category', expr("CASE WHEN count<10 THEN 'under' WHEN count>=10 THEN 'upper' END"))
df4.show()

+--------------------+-------------------+-----+--------+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|Category|
+--------------------+-------------------+-----+--------+
|       United States|            Romania|   15|   upper|
|       United States|            Croatia|    1|   under|
|       United States|            Ireland|  344|   upper|
|               Egypt|      United States|   15|   upper|
|       United States|              India|   62|   upper|
|       United States|          Singapore|    1|   under|
|       United States|            Grenada|   62|   upper|
|          Costa Rica|      United States|  588|   upper|
|             Senegal|      United States|   40|   upper|
|             Moldova|      United States|    1|   under|
|       United States|       Sint Maarten|  325|   upper|
|       United States|   Marshall Islands|   39|   upper|
|              Guyana|      United States|   64|   upper|
|               Malta|      United States|    1|   under|
|            A

In [42]:
# case where 카운트 10이하이면 under, 이상 upper로 변환 > category 컬럼 추가 
df3.createOrReplaceTempView("df3") 
query = """
SELECT *, CASE WHEN count >= 10 THEN 'upper' ELSE 'under' END as category FROM df3 
"""
spark.sql(query).show(10)

+-----------------+-------------------+-----+-------------+--------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withincountry|category|
+-----------------+-------------------+-----+-------------+--------+
|    United States|            Romania|   15|        false|   upper|
|    United States|            Croatia|    1|        false|   under|
|    United States|            Ireland|  344|        false|   upper|
|            Egypt|      United States|   15|        false|   upper|
|    United States|              India|   62|        false|   upper|
|    United States|          Singapore|    1|        false|   under|
|    United States|            Grenada|   62|        false|   upper|
|       Costa Rica|      United States|  588|        false|   upper|
|          Senegal|      United States|   40|        false|   upper|
|          Moldova|      United States|    1|        false|   under|
+-----------------+-------------------+-----+-------------+--------+
only showing top 10 rows



In [43]:
# DataFrame의 select(), where(), filter() 트랜스포메이션
# show(), count() 액션

In [60]:
spark.stop()

In [46]:
# 집계함수

In [61]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("spark-sql").getOrCreate()

In [62]:
import os
print(os.path.abspath('emp.csv'))

/home/jovyan/work/practice_virtual-environment/emp.csv


In [69]:
# emp_df, dept_df
emp_df = spark.read.format('csv')\
        .option('header','true')\
        .option('inferSchema','true')\
        .load("/home/jovyan/work/practice_virtual-environment/emp.csv")
dept_df = spark.read.format('csv')\
        .option('header','true')\
        .option('inferSchema','true')\
        .load("/home/jovyan/work/practice_virtual-environment/dept.csv")

In [70]:
emp_df.show(3)

+-----+-----+--------+----+----------+----+----+------+
|empno|ename|     job| mgr|  hiredate| sal|comm|deptno|
+-----+-----+--------+----+----------+----+----+------+
| 7369|SMITH|   CLERK|7902|1980-12-17| 800|NULL|    20|
| 7499|ALLEN|SALESMAN|7698|1981-02-20|1600| 300|    30|
| 7521| WARD|SALESMAN|7698|1981-02-22|1250| 500|    30|
+-----+-----+--------+----+----------+----+----+------+
only showing top 3 rows



In [71]:
dept_df.show(3)

+------+----------+--------+
|deptno|     dname|     loc|
+------+----------+--------+
|    10|ACCOUNTING|NEW YORK|
|    20|  RESEARCH|  DALLAS|
|    30|     SALES| CHICAGO|
+------+----------+--------+
only showing top 3 rows



In [72]:
emp_df.printSchema()

root
 |-- empno: integer (nullable = true)
 |-- ename: string (nullable = true)
 |-- job: string (nullable = true)
 |-- mgr: integer (nullable = true)
 |-- hiredate: date (nullable = true)
 |-- sal: integer (nullable = true)
 |-- comm: integer (nullable = true)
 |-- deptno: integer (nullable = true)



In [73]:
dept_df.printSchema()

root
 |-- deptno: integer (nullable = true)
 |-- dname: string (nullable = true)
 |-- loc: string (nullable = true)



In [77]:
emp_df.select('ename','deptno').show()

+------+------+
| ename|deptno|
+------+------+
| SMITH|    20|
| ALLEN|    30|
|  WARD|    30|
| JONES|    20|
|MARTIN|    30|
| BLAKE|    30|
| CLARK|    10|
| SCOTT|    20|
|  KING|    10|
|TURNER|    30|
| ADAMS|    20|
| JAMES|    30|
|  FORD|    20|
|MILLER|    10|
|  JACK|    70|
+------+------+



In [79]:
emp_df.select('*').where('deptno=20').show()

+-----+-----+-------+----+----------+----+----+------+
|empno|ename|    job| mgr|  hiredate| sal|comm|deptno|
+-----+-----+-------+----+----------+----+----+------+
| 7369|SMITH|  CLERK|7902|1980-12-17| 800|NULL|    20|
| 7566|JONES|MANAGER|7839|1981-04-02|2975|NULL|    20|
| 7788|SCOTT|ANALYST|7566|1987-04-19|3000|NULL|    20|
| 7876|ADAMS|  CLERK|7788|1987-05-23|1100|NULL|    20|
| 7902| FORD|ANALYST|7566|1981-12-03|3000|NULL|    20|
+-----+-----+-------+----+----------+----+----+------+



In [83]:
emp_df.selectExpr('count(*)').show()

+--------+
|count(1)|
+--------+
|      15|
+--------+



In [88]:
from pyspark.sql.functions import countDistinct
emp_df.select(countDistinct('job')).show()

+-------------------+
|count(DISTINCT job)|
+-------------------+
|                  5|
+-------------------+



In [89]:
from pyspark.sql.functions import approx_count_distinct
emp_df.select(approx_count_distinct('job',0.1)).show()

+--------------------------+
|approx_count_distinct(job)|
+--------------------------+
|                         5|
+--------------------------+



In [92]:
# first, last, min, max, sum, avg -> (expr : sql 문장 x ), function으로 처리
#df.select( count('sal'))

In [95]:
from pyspark.sql.functions import first
emp_df.select( first('sal')).show()

+----------+
|first(sal)|
+----------+
|       800|
+----------+



In [96]:
from pyspark.sql.functions import last
emp_df.select( last('job')).show()

+---------+
|last(job)|
+---------+
|    CLERK|
+---------+



In [97]:
from pyspark.sql.functions import min
emp_df.select( min('sal')).show()

+--------+
|min(sal)|
+--------+
|     800|
+--------+



In [98]:
from pyspark.sql.functions import max
emp_df.select( max('mgr')).show()

+--------+
|max(mgr)|
+--------+
|    7902|
+--------+



In [99]:
from pyspark.sql.functions import sum
emp_df.select( sum('sal')).show()

+--------+
|sum(sal)|
+--------+
|   32225|
+--------+



In [100]:
from pyspark.sql.functions import avg
emp_df.select( avg('sal')).show()

+------------------+
|          avg(sal)|
+------------------+
|2148.3333333333335|
+------------------+



In [102]:
from pyspark.sql.functions import col
emp_df.select(sum(col('sal'))).show()

+--------+
|sum(sal)|
+--------+
|   32225|
+--------+



In [103]:
emp_df.selectExpr(' sum(distinct sal) ').show()

+-----------------+
|sum(DISTINCT sal)|
+-----------------+
|            27975|
+-----------------+



In [106]:
# total_salary/ total_transaction, avg_salary, mean_salary
from pyspark.sql.functions import count, mean
emp_df.select((sum('sal')/count('sal')).alias('계산'), avg('sal'), mean('sal').alias('mean(sal)')).show()

+------------------+------------------+------------------+
|              계산|          avg(sal)|         mean(sal)|
+------------------+------------------+------------------+
|2148.3333333333335|2148.3333333333335|2148.3333333333335|
+------------------+------------------+------------------+



In [110]:
# 강사님 작성 코드
emp_df.select(
    count('sal').alias('total_transaction'),
    sum('sal').alias('total_salary'),
    avg('sal').alias('avg_salary'),
    mean('sal').alias('mean_salary')
).selectExpr(
    'total_salary/total_transaction as avg_trans',
    'avg_salary',
    'mean_salary'
).show()

+------------------+------------------+------------------+
|         avg_trans|        avg_salary|       mean_salary|
+------------------+------------------+------------------+
|2148.3333333333335|2148.3333333333335|2148.3333333333335|
+------------------+------------------+------------------+



In [111]:
# 그룹화
emp_df.groupBy('job').count().show()

+---------+-----+
|      job|count|
+---------+-----+
|  ANALYST|    2|
| SALESMAN|    4|
|    CLERK|    5|
|  MANAGER|    3|
|PRESIDENT|    1|
+---------+-----+



In [115]:
# select job, 
    #count(job),
    #sum(sal)
#from df3
#groub by job

group_df = emp_df.groupBy('job').agg(
    count('job').alias('qty'),
    expr('count(job)'),
    sum('sal')
)
group_df.show()

+---------+---+----------+--------+
|      job|qty|count(job)|sum(sal)|
+---------+---+----------+--------+
|  ANALYST|  2|         2|    6000|
| SALESMAN|  4|         4|    5600|
|    CLERK|  5|         5|    7350|
|  MANAGER|  3|         3|    8275|
|PRESIDENT|  1|         1|    5000|
+---------+---+----------+--------+



In [124]:
# sal의 평균SAL_AVG, 표준편차SAL_STDDEV를 job별로 게산해서 출력
from pyspark.sql.functions import stddev, round
emp_df.groupBy('job').agg(
    round(avg('sal'),2).alias('SAL_AVG'),
    round(stddev('sal'),2).alias('SAL_STDDEV')
).show()   

+---------+-------+----------+
|      job|SAL_AVG|SAL_STDDEV|
+---------+-------+----------+
|  ANALYST| 3000.0|       0.0|
| SALESMAN| 1400.0|    177.95|
|    CLERK| 1470.0|    984.63|
|  MANAGER|2758.33|    274.24|
|PRESIDENT| 5000.0|      NULL|
+---------+-------+----------+



In [130]:
# 급여 TOP10 구하기
from pyspark.sql.functions import desc
emp_df.orderBy(desc('sal')).show(10)
#  강사님 코드
# emp_df.orderBy(emp_df.sal.desc()).limit(10).show()

+-----+------+---------+----+----------+----+----+------+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|
+-----+------+---------+----+----------+----+----+------+
| 7839|  KING|PRESIDENT|NULL|1981-11-17|5000|NULL|    10|
| 9292|  JACK|    CLERK|7782|1982-01-23|3200|NULL|    70|
| 7788| SCOTT|  ANALYST|7566|1987-04-19|3000|NULL|    20|
| 7902|  FORD|  ANALYST|7566|1981-12-03|3000|NULL|    20|
| 7566| JONES|  MANAGER|7839|1981-04-02|2975|NULL|    20|
| 7698| BLAKE|  MANAGER|7839|1981-05-01|2850|NULL|    30|
| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|NULL|    10|
| 7499| ALLEN| SALESMAN|7698|1981-02-20|1600| 300|    30|
| 7844|TURNER| SALESMAN|7698|1981-09-08|1500|   0|    30|
| 7934|MILLER|    CLERK|7782|1982-01-23|1300|NULL|    10|
+-----+------+---------+----+----------+----+----+------+
only showing top 10 rows



In [140]:
# 윈도우 함수
from pyspark.sql.window import Window
from pyspark.sql.functions import rank
WindowSpec = Window.orderBy(desc('sal'))
salALLRank = rank().over(WindowSpec)
salALLRank

Column<'RANK() OVER (ORDER BY sal DESC NULLS LAST unspecifiedframe$())'>

In [141]:
emp_df.show(10)

+-----+------+---------+----+----------+----+----+------+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|
+-----+------+---------+----+----------+----+----+------+
| 7369| SMITH|    CLERK|7902|1980-12-17| 800|NULL|    20|
| 7499| ALLEN| SALESMAN|7698|1981-02-20|1600| 300|    30|
| 7521|  WARD| SALESMAN|7698|1981-02-22|1250| 500|    30|
| 7566| JONES|  MANAGER|7839|1981-04-02|2975|NULL|    20|
| 7654|MARTIN| SALESMAN|7698|1981-09-28|1250|1400|    30|
| 7698| BLAKE|  MANAGER|7839|1981-05-01|2850|NULL|    30|
| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|NULL|    10|
| 7788| SCOTT|  ANALYST|7566|1987-04-19|3000|NULL|    20|
| 7839|  KING|PRESIDENT|NULL|1981-11-17|5000|NULL|    10|
| 7844|TURNER| SALESMAN|7698|1981-09-08|1500|   0|    30|
+-----+------+---------+----+----------+----+----+------+
only showing top 10 rows



In [146]:
emp_df.withColumn('salary_rank', salALLRank).show(10)

+-----+------+---------+----+----------+----+----+------+-----------+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|salary_rank|
+-----+------+---------+----+----------+----+----+------+-----------+
| 7839|  KING|PRESIDENT|NULL|1981-11-17|5000|NULL|    10|          1|
| 9292|  JACK|    CLERK|7782|1982-01-23|3200|NULL|    70|          2|
| 7788| SCOTT|  ANALYST|7566|1987-04-19|3000|NULL|    20|          3|
| 7902|  FORD|  ANALYST|7566|1981-12-03|3000|NULL|    20|          3|
| 7566| JONES|  MANAGER|7839|1981-04-02|2975|NULL|    20|          5|
| 7698| BLAKE|  MANAGER|7839|1981-05-01|2850|NULL|    30|          6|
| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|NULL|    10|          7|
| 7499| ALLEN| SALESMAN|7698|1981-02-20|1600| 300|    30|          8|
| 7844|TURNER| SALESMAN|7698|1981-09-08|1500|   0|    30|          9|
| 7934|MILLER|    CLERK|7782|1982-01-23|1300|NULL|    10|         10|
+-----+------+---------+----+----------+----+----+------+-----------+
only showing top 10 

In [166]:
# 직무별로 rank 작성
# window.partitionBy()
# job_rank_df 작성
from pyspark.sql.functions import row_number
job_rank_spec = Window.partitionBy('job').orderBy(col('sal').desc())
job_rank_df = emp_df.withColumn('job_rank', row_number().over(job_rank_spec))
job_rank_df.show()

+-----+------+---------+----+----------+----+----+------+--------+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|job_rank|
+-----+------+---------+----+----------+----+----+------+--------+
| 7788| SCOTT|  ANALYST|7566|1987-04-19|3000|NULL|    20|       1|
| 7902|  FORD|  ANALYST|7566|1981-12-03|3000|NULL|    20|       2|
| 9292|  JACK|    CLERK|7782|1982-01-23|3200|NULL|    70|       1|
| 7934|MILLER|    CLERK|7782|1982-01-23|1300|NULL|    10|       2|
| 7876| ADAMS|    CLERK|7788|1987-05-23|1100|NULL|    20|       3|
| 7900| JAMES|    CLERK|7698|1981-12-03| 950|NULL|    30|       4|
| 7369| SMITH|    CLERK|7902|1980-12-17| 800|NULL|    20|       5|
| 7566| JONES|  MANAGER|7839|1981-04-02|2975|NULL|    20|       1|
| 7698| BLAKE|  MANAGER|7839|1981-05-01|2850|NULL|    30|       2|
| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|NULL|    10|       3|
| 7839|  KING|PRESIDENT|NULL|1981-11-17|5000|NULL|    10|       1|
| 7499| ALLEN| SALESMAN|7698|1981-02-20|1600| 300|    30|     

In [188]:
# 부셔별 순위 
job_rank_df.select("deptno", "job", "ename", "job_rank").orderBy("deptno", "job_rank").show()

+------+---------+------+--------+
|deptno|      job| ename|job_rank|
+------+---------+------+--------+
|    10|PRESIDENT|  KING|       1|
|    10|    CLERK|MILLER|       2|
|    10|  MANAGER| CLARK|       3|
|    20|  ANALYST| SCOTT|       1|
|    20|  MANAGER| JONES|       1|
|    20|  ANALYST|  FORD|       2|
|    20|    CLERK| ADAMS|       3|
|    20|    CLERK| SMITH|       5|
|    30| SALESMAN| ALLEN|       1|
|    30|  MANAGER| BLAKE|       2|
|    30| SALESMAN|TURNER|       2|
|    30| SALESMAN|  WARD|       3|
|    30|    CLERK| JAMES|       4|
|    30| SALESMAN|MARTIN|       4|
|    70|    CLERK|  JACK|       1|
+------+---------+------+--------+



In [180]:
# 누적 급여 sum('sal').over()
emp_df.groupBy("job").agg(sum("sal").alias("total_sal")).show()


+---------+---------+
|      job|total_sal|
+---------+---------+
|  ANALYST|     6000|
| SALESMAN|     5600|
|    CLERK|     7350|
|  MANAGER|     8275|
|PRESIDENT|     5000|
+---------+---------+



In [182]:
# 부서별 누적 급여 
emp_df.groupBy("deptno").agg(sum("sal").alias("total_sal")).show()

+------+---------+
|deptno|total_sal|
+------+---------+
|    20|    10875|
|    10|     8750|
|    70|     3200|
|    30|     9400|
+------+---------+



In [ ]:
# 부서별 평균 금여와 직원 개별 급여 비교
dept_avg_spec = Window.avg('sal').
dept_avg_df = emp_df.withColumn('job_rank', row_number().over(job_rank_spec))

In [192]:
# 부서별 직업별 소계
emp_df.groupBy('deptno', 'job').agg(count('*'),sum('sal'))\
    .orderBy('deptno', 'job').show()

+------+---------+--------+--------+
|deptno|      job|count(1)|sum(sal)|
+------+---------+--------+--------+
|    10|    CLERK|       1|    1300|
|    10|  MANAGER|       1|    2450|
|    10|PRESIDENT|       1|    5000|
|    20|  ANALYST|       2|    6000|
|    20|    CLERK|       2|    1900|
|    20|  MANAGER|       1|    2975|
|    30|    CLERK|       1|     950|
|    30|  MANAGER|       1|    2850|
|    30| SALESMAN|       4|    5600|
|    70|    CLERK|       1|    3200|
+------+---------+--------+--------+



In [193]:
# 부서별 직업별 소계
emp_df.cube('deptno', 'job').agg(count('*'),sum('sal'))\
    .orderBy('deptno', 'job').show() 

+------+---------+--------+--------+
|deptno|      job|count(1)|sum(sal)|
+------+---------+--------+--------+
|  NULL|     NULL|      15|   32225|
|  NULL|  ANALYST|       2|    6000|
|  NULL|    CLERK|       5|    7350|
|  NULL|  MANAGER|       3|    8275|
|  NULL|PRESIDENT|       1|    5000|
|  NULL| SALESMAN|       4|    5600|
|    10|     NULL|       3|    8750|
|    10|    CLERK|       1|    1300|
|    10|  MANAGER|       1|    2450|
|    10|PRESIDENT|       1|    5000|
|    20|     NULL|       5|   10875|
|    20|  ANALYST|       2|    6000|
|    20|    CLERK|       2|    1900|
|    20|  MANAGER|       1|    2975|
|    30|     NULL|       6|    9400|
|    30|    CLERK|       1|     950|
|    30|  MANAGER|       1|    2850|
|    30| SALESMAN|       4|    5600|
|    70|     NULL|       1|    3200|
|    70|    CLERK|       1|    3200|
+------+---------+--------+--------+



In [196]:
# 부서별 직업별 소계
emp_df.cube('deptno', 'job').agg(count('*'),avg('sal'))\
    .orderBy('deptno', 'job').show() # 평균급

+------+---------+--------+------------------+
|deptno|      job|count(1)|          avg(sal)|
+------+---------+--------+------------------+
|  NULL|     NULL|      15|2148.3333333333335|
|  NULL|  ANALYST|       2|            3000.0|
|  NULL|    CLERK|       5|            1470.0|
|  NULL|  MANAGER|       3|2758.3333333333335|
|  NULL|PRESIDENT|       1|            5000.0|
|  NULL| SALESMAN|       4|            1400.0|
|    10|     NULL|       3|2916.6666666666665|
|    10|    CLERK|       1|            1300.0|
|    10|  MANAGER|       1|            2450.0|
|    10|PRESIDENT|       1|            5000.0|
|    20|     NULL|       5|            2175.0|
|    20|  ANALYST|       2|            3000.0|
|    20|    CLERK|       2|             950.0|
|    20|  MANAGER|       1|            2975.0|
|    30|     NULL|       6|1566.6666666666667|
|    30|    CLERK|       1|             950.0|
|    30|  MANAGER|       1|            2850.0|
|    30| SALESMAN|       4|            1400.0|
|    70|     

In [195]:
# 부서별 직업별 소계
emp_df.cube('deptno', 'job').agg(count('*'),max('sal'))\
    .orderBy('deptno', 'job').show() # 최대급

+------+---------+--------+--------+
|deptno|      job|count(1)|max(sal)|
+------+---------+--------+--------+
|  NULL|     NULL|      15|    5000|
|  NULL|  ANALYST|       2|    3000|
|  NULL|    CLERK|       5|    3200|
|  NULL|  MANAGER|       3|    2975|
|  NULL|PRESIDENT|       1|    5000|
|  NULL| SALESMAN|       4|    1600|
|    10|     NULL|       3|    5000|
|    10|    CLERK|       1|    1300|
|    10|  MANAGER|       1|    2450|
|    10|PRESIDENT|       1|    5000|
|    20|     NULL|       5|    3000|
|    20|  ANALYST|       2|    3000|
|    20|    CLERK|       2|    1100|
|    20|  MANAGER|       1|    2975|
|    30|     NULL|       6|    2850|
|    30|    CLERK|       1|     950|
|    30|  MANAGER|       1|    2850|
|    30| SALESMAN|       4|    1600|
|    70|     NULL|       1|    3200|
|    70|    CLERK|       1|    3200|
+------+---------+--------+--------+



In [194]:
# 부서별 직업별 소계
emp_df.cube('deptno', 'job').agg(count('*'),min('sal'))\
    .orderBy('deptno', 'job').show() # 최소급

+------+---------+--------+--------+
|deptno|      job|count(1)|min(sal)|
+------+---------+--------+--------+
|  NULL|     NULL|      15|     800|
|  NULL|  ANALYST|       2|    3000|
|  NULL|    CLERK|       5|     800|
|  NULL|  MANAGER|       3|    2450|
|  NULL|PRESIDENT|       1|    5000|
|  NULL| SALESMAN|       4|    1250|
|    10|     NULL|       3|    1300|
|    10|    CLERK|       1|    1300|
|    10|  MANAGER|       1|    2450|
|    10|PRESIDENT|       1|    5000|
|    20|     NULL|       5|     800|
|    20|  ANALYST|       2|    3000|
|    20|    CLERK|       2|     800|
|    20|  MANAGER|       1|    2975|
|    30|     NULL|       6|     950|
|    30|    CLERK|       1|     950|
|    30|  MANAGER|       1|    2850|
|    30| SALESMAN|       4|    1250|
|    70|     NULL|       1|    3200|
|    70|    CLERK|       1|    3200|
+------+---------+--------+--------+



In [ ]:
# 직원, 부서 조인 

In [197]:
emp_df.printSchema()

root
 |-- empno: integer (nullable = true)
 |-- ename: string (nullable = true)
 |-- job: string (nullable = true)
 |-- mgr: integer (nullable = true)
 |-- hiredate: date (nullable = true)
 |-- sal: integer (nullable = true)
 |-- comm: integer (nullable = true)
 |-- deptno: integer (nullable = true)



In [198]:
dept_df.printSchema()

root
 |-- deptno: integer (nullable = true)
 |-- dname: string (nullable = true)
 |-- loc: string (nullable = true)



In [199]:
emp_dept_df = emp_df.join(dept_df, emp_df['deptno']==dept_df['deptno'])
emp_dept_df.show()

+-----+------+---------+----+----------+----+----+------+------+----------+--------+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|deptno|     dname|     loc|
+-----+------+---------+----+----------+----+----+------+------+----------+--------+
| 7369| SMITH|    CLERK|7902|1980-12-17| 800|NULL|    20|    20|  RESEARCH|  DALLAS|
| 7499| ALLEN| SALESMAN|7698|1981-02-20|1600| 300|    30|    30|     SALES| CHICAGO|
| 7521|  WARD| SALESMAN|7698|1981-02-22|1250| 500|    30|    30|     SALES| CHICAGO|
| 7566| JONES|  MANAGER|7839|1981-04-02|2975|NULL|    20|    20|  RESEARCH|  DALLAS|
| 7654|MARTIN| SALESMAN|7698|1981-09-28|1250|1400|    30|    30|     SALES| CHICAGO|
| 7698| BLAKE|  MANAGER|7839|1981-05-01|2850|NULL|    30|    30|     SALES| CHICAGO|
| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|NULL|    10|    10|ACCOUNTING|NEW YORK|
| 7788| SCOTT|  ANALYST|7566|1987-04-19|3000|NULL|    20|    20|  RESEARCH|  DALLAS|
| 7839|  KING|PRESIDENT|NULL|1981-11-17|5000|NULL|    10|    10|A

In [200]:
join_df = emp_df.join(dept_df, on='deptno', how='inner')
join_df.select('ename', 'deptno', 'dname').show()

+------+------+----------+
| ename|deptno|     dname|
+------+------+----------+
| SMITH|    20|  RESEARCH|
| ALLEN|    30|     SALES|
|  WARD|    30|     SALES|
| JONES|    20|  RESEARCH|
|MARTIN|    30|     SALES|
| BLAKE|    30|     SALES|
| CLARK|    10|ACCOUNTING|
| SCOTT|    20|  RESEARCH|
|  KING|    10|ACCOUNTING|
|TURNER|    30|     SALES|
| ADAMS|    20|  RESEARCH|
| JAMES|    30|     SALES|
|  FORD|    20|  RESEARCH|
|MILLER|    10|ACCOUNTING|
+------+------+----------+



In [204]:
spark.stop()